# 6. 업체별 참여공사 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "업체 목록"
### APIKEY 불러오기 ###
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/업체 목록/업체 목록.csv'

In [2]:
targetPath = targetData.저장폴더.values[0]

In [3]:
conList = pd.read_csv(targetPath, encoding="ms949")

In [4]:
brnList = conList.brn.drop_duplicates().tolist()

### 6-1. 업체 목록 내 brn 가져오기

In [5]:
### 기본정보 설정 및 파라미터 설정 ###
SITENAME = "건설사업정보시스템"
DATANAME= "업체별 참여공사 목록"
# APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###

In [7]:
newParam = brnList.copy()
# if newParam==[]:
    # mode=2
    # print("{} 정보 new 모드 {} ".format(DATANAME, mode))

In [8]:
orgParam = []
try:
    # 기존 데이터 및 파라미터 정보 불러오기
    # 리스트로 불러오기로 변경 0425
    orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)[0]
    outData = cf.loaddata(SITENAME,DATANAME,SERVICENAME)
## 파일이 없는경우
except Exception as e:
    orgParam = []
    print(e)

ptlcmnoList = list( set(newParam) - set(orgParam) )
# 모드: 0=종료 1=append 2=새로생성
mode = 2

# 업데이트 할 내용이 없으면 종료
if ptlcmnoList == []:
    mode = 0
    print("{} 정보 quit모드 {} ".format(DATANAME, mode))
    quit()
# orgParam == []
elif orgParam == []:
    mode = 2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))
# 이외에는 append 모드
else:
    mode = 1
    print("{} 정보 append 모드 {} ".format(DATANAME, mode))

[Errno 2] No such file or directory: '../output\\건설사업정보시스템\\업체별 참여공사 목록\\selectIoCmProjCorpPartList.pickle'
업체별 참여공사 목록 정보 new 모드 2 


In [9]:
### 개별 코드 작업 영역 ###
### baseparam은 수동설정필요

ptlcmnoListLen = len(ptlcmnoList)
resultDfMerged = pd.DataFrame()
pageList = [] 

# ptlcmnoListLen = len(ptlcmnoList)
APICALL = 0
resultDfMerged = pd.DataFrame()
for i in range(0, ptlcmnoListLen):
    BASEPARAM={"serviceKey":APIKEY[0], "type":"json", "brn": ptlcmnoList[i]} #변경 apiNo -> 0
    # scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN)
    scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN,APIKEY,APICALL) #추가
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2]  #추가
    resultDfMerged = resultDfMerged.append(resultDf)

1 page scraping start apicall iter: 1 / used 11067238-08C1-4FDE-857B-4C865C85F2A7
totalCount 77
0 no pageNo
dataframe업체별 참여공사 목록, param:{'serviceKey': '11067238-08C1-4FDE-857B-4C865C85F2A7', 'type': 'json', 'brn': 1278152705} rows: 14 completed
1 page scraping start apicall iter: 2 / used 9D486CED-81E4-4884-96FF-BB1F14A9223B
totalCount 2
0 no pageNo
dataframe업체별 참여공사 목록, param:{'serviceKey': '9D486CED-81E4-4884-96FF-BB1F14A9223B', 'type': 'json', 'brn': 4088115201} rows: 14 completed
1 page scraping start apicall iter: 3 / used 11067238-08C1-4FDE-857B-4C865C85F2A7
totalCount 1
0 no pageNo
dataframe업체별 참여공사 목록, param:{'serviceKey': '11067238-08C1-4FDE-857B-4C865C85F2A7', 'type': 'json', 'brn': 4088102915} rows: 14 completed
1 page scraping start apicall iter: 4 / used 9D486CED-81E4-4884-96FF-BB1F14A9223B
totalCount 1
0 no pageNo
dataframe업체별 참여공사 목록, param:{'serviceKey': '9D486CED-81E4-4884-96FF-BB1F14A9223B', 'type': 'json', 'brn': 3018149905} rows: 14 completed
1 page scraping start a

In [10]:
del resultDfMerged["linkyn"]

In [11]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME, mode)

resultDfMerged.shape

업체별 참여공사 목록 save compled


(4638, 13)

In [12]:
# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
ptlcmnoList = orgParam + ptlcmnoList

paramList = [ptlcmnoList,pageList]
# 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)